# 1 Import Libraries


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

# 2 Load the Data

In [4]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

print("Train shape", train.shape)
print("Test shape", test.shape)

Train shape (1460, 81)
Test shape (1459, 80)


# 4 Combining the train test for preprocessing

In [6]:
train_ID=train["Id"]
test_ID=test["Id"]

train.drop(["Id"], axis=1, inplace=True)
test.drop(["Id"], axis=1, inplace=True)

n_train=train.shape[0]
combined=pd.concat([train,test],ignore_index=True)

print("Combined shape:", combined.shape)

Combined shape: (2919, 80)


In [7]:
combined.loc[:n_train-1,'SalePrice']=np.log(combined.loc[:n_train-1,'SalePrice'])

In [8]:
missing=combined.isnull().sum().sort_values(ascending=False)
missing = missing[missing>0]
print(missing)

PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
MasVnrType      1766
SalePrice       1459
FireplaceQu     1420
LotFrontage      486
GarageYrBlt      159
GarageCond       159
GarageQual       159
GarageFinish     159
GarageType       157
BsmtExposure      82
BsmtCond          82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrArea        23
MSZoning           4
Utilities          2
Functional         2
BsmtFullBath       2
BsmtHalfBath       2
BsmtFinSF2         1
BsmtFinSF1         1
TotalBsmtSF        1
BsmtUnfSF          1
SaleType           1
KitchenQual        1
GarageCars         1
GarageArea         1
Exterior2nd        1
Exterior1st        1
Electrical         1
dtype: int64
